In [1]:
import warnings 
warnings.filterwarnings('ignore')
import time 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import tensorflow as tf 
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def get_score(model, X, y_true):
    y_pred = model.predict(X)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_precision_error(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Loading augmented data

In [3]:
# Optimal which_model(100, number =30, batch_size=128, l1=512, l2=128, l3=16, lr=1e-4, step_log=10)

In [4]:
#test  = pd.read_pickle('dataFrames/test_OneHotEncoding.pkl')
#train = pd.read_pickle('dataFrames/train_OneHotEncoding.pkl')
test   = pd.read_pickle('dataFrames/test_OneHotEncoding_new_June14th.pkl')
train  = pd.read_pickle('dataFrames/train_OneHotEncoding_new_June14th.pkl')

#test    = test.drop(['y_avg_lgb_xgb', 'y_gboost', 'y_xgb', 'y_bag', 'y_knn', 'y_dec', 'y_lgb'], axis=1)
#train   = train.drop(['y_avg_lgb_xgb', 'y_gboost', 'y_xgb', 'y_bag', 'y_knn', 'y_dec', 'y_lgb'], axis=1)

continuous_cols = ['destinationLatitude', 'destinationLongitude', 'distanceKM', 'sourceLatitude', 
                   'sourceLongitude', 'taxiDurationMin', 'weight', 'source', 'destination', 
                   'y_avg_lgb_xgb','y_gboost', 'y_xgb', 'y_bag', 'y_knn', 'y_dec', 'y_lgb' ]

#continuous_cols = [ 'y_gboost', 'y_xgb', 'y_lgb', 'y_bag', 'distanceKM', 
#                   'taxiDurationMin', 'weight', 'source', 'destination']

categorical_cols = train.columns.drop(continuous_cols + ['ID', 'price']).tolist()

NOM = train[categorical_cols].shape[1]
renaming_dict = dict(zip(train[categorical_cols].columns, [str(x) for x in list(range(NOM)) ]))

train_renamed = train[categorical_cols].rename(columns=renaming_dict)
test_renamed  = test[categorical_cols].rename(columns=renaming_dict)

for column in continuous_cols:
    train_renamed[column] = train[column]
    test_renamed[column] = test[column]
    
test_renamed['ID']   = test['ID']
train_renamed['ID'] = train['ID']
test_renamed['price'] = test['price']
train_renamed['price'] = train['price']

X_train, X_val = train_test_split(train_renamed, test_size=0.2, random_state=42)
X_train.head()

0  1  2  3  4  5  6  7  8  9     ...      destination  y_avg_lgb_xgb  \
6987   0  0  0  0  0  0  1  0  0  0     ...      1909.126148   2.478849e+07   
42079  0  0  0  0  0  0  0  0  0  1     ...      1550.413580   1.262995e+07   
36862  0  0  0  0  0  0  0  0  0  0     ...      1835.223732   7.573594e+06   
28387  0  0  0  0  0  0  0  0  0  0     ...      1688.080124   3.345341e+06   
35474  0  0  0  0  1  0  0  0  0  0     ...      1748.831805   3.207776e+06   

           y_gboost        y_xgb         y_bag      y_knn         y_dec  \
6987   2.328065e+07  24841406.00  2.457702e+07  9000000.0  2.483060e+07   
42079  9.319747e+06   9286450.00  1.573842e+07  7500000.0  8.900000e+06   
36862  7.651516e+06   7973744.50  7.752710e+06  7000000.0  5.318050e+06   
28387  2.828410e+06   8076054.00  3.173603e+06  2470000.0  3.644693e+06   
35474  2.005464e+06   2953551.75  2.637220e+06  2915000.0  2.141866e+06   

              y_lgb           ID       price  
6987   2.434387e+07  52642600134  24500000.0  
42079  9.594753e+06  52386856932   9000000.0  
36862  7.913213e+06  19056360759   8000000.0  
28387  3.762905e+06  54034736201   3450000.0  
35474  2.794782e+06  64195974128   3300000.0  

[5 rows x 93 columns]

In [5]:
print("%.2f" % np.mean(100*np.abs((train['y_gboost'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_avg_lgb_xgb'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_xgb'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_lgb'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_bag'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_dec'] - train['price']) / train['price'])))
print("%.2f" % np.mean(100*np.abs((train['y_knn'] - train['price']) / train['price'])))

17.80
17.98
18.39
18.49
19.83
24.41
26.86


# TensorFlow combination  

In [6]:
BATCH_SIZE          = 128
TRAIN_EPOCHS        = 450
HIDDEN_LAYER_1_SIZE = 512
HIDDEN_LAYER_2_SIZE = 512
HIDDEN_LAYER_3_SIZE = 16
lr                  = 1e-4
USE_ALL_FEATURES    = True

In [7]:
def make_model(features, labels, mode, params, config):
    input_layer = tf.feature_column.input_layer(features=features, feature_columns=feature_columns)
    global_step = tf.train.get_or_create_global_step()
    x = tf.layers.dense(inputs=input_layer, units=HIDDEN_LAYER_1_SIZE, activation=tf.nn.relu, name="first_layer")
    x = tf.layers.dropout(inputs=x,name="first_dropout")
    x = tf.layers.dense(inputs=x, units=HIDDEN_LAYER_2_SIZE, activation=tf.nn.relu, name="second_layer")
    x = tf.layers.dropout(inputs=x,name="second_dropout")
    x = tf.layers.dense(inputs=x, units=HIDDEN_LAYER_3_SIZE, activation=tf.nn.relu, name="third_layer")
    predictions = tf.contrib.layers.fully_connected(inputs=x, num_outputs=1)
    if mode == tf.estimator.ModeKeys.PREDICT :
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss  = tf.reduce_mean(tf.abs(tf.divide(predictions-labels,labels))) 
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, loss=loss)
    else:
        loss  = tf.reduce_mean(tf.abs(tf.divide(predictions-labels,labels))) 
        tf.summary.scalar("Loss", loss)
        optimizer = tf.train.AdamOptimizer(learning_rate=params.learning_rate)
        train_op = optimizer.minimize(loss, global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [8]:
feature_columns = set()

if USE_ALL_FEATURES:
    for col in categorical_cols:
        col_feat = tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_identity(renaming_dict[col], 2),2)
        feature_columns.add(col_feat)

for cont in continuous_cols:
    col_feat = tf.feature_column.numeric_column(cont)
    feature_columns.add(col_feat)
    
print(len(feature_columns))

91


In [9]:
def input_fn(df, pred = False, use_all_features = USE_ALL_FEATURES):
        
    useful_fueatures = list()
    
    if use_all_features:
        for col in categorical_cols:
            useful_fueatures.append(np.array(df[renaming_dict[col]].values, dtype=np.int32))

    for cont in continuous_cols:
        useful_fueatures.append(np.array(df[cont].values, dtype=np.float32))    
    
    if pred: 
        train_number = 1
        batch_number = 1
    else:
        useful_fueatures.append(np.array(df["price"].values, dtype=np.float32))
        train_number = TRAIN_EPOCHS
        batch_number = BATCH_SIZE
        
    A = tf.train.slice_input_producer(
        tensor_list=useful_fueatures,
        num_epochs=train_number,
        shuffle= not pred,
        capacity=BATCH_SIZE * 5
    )

    dataset_dict = dict()
    
    if use_all_features:
        for i in range(len(A)):
            if i < len(categorical_cols):
                dataset_dict[renaming_dict[categorical_cols[i]]] = A[i]
            elif i < len(categorical_cols) + len(continuous_cols):
                dataset_dict[continuous_cols[i-len(categorical_cols)]] = A[i]
    else:
        for i in range(len(A)):
            if i < len(continuous_cols):
                dataset_dict[continuous_cols[i]] = A[i]
            
    if not pred:
        dataset_dict['labels'] = A[-1]
            
    batch_dict = tf.train.batch(
        dataset_dict,
        batch_number,
   )

    if pred == False:
        batch_labels = batch_dict.pop('labels')
        return batch_dict, tf.reshape(batch_labels, [-1, 1]) 
    else:
        return batch_dict 

In [11]:
hparams = tf.contrib.training.HParams(learning_rate=lr)
rconfig = tf.estimator.RunConfig(log_step_count_steps = 10, save_summary_steps = 1)
estimator_val = tf.estimator.Estimator(model_fn=make_model, params=hparams, config = rconfig)
estimator_val.train(input_fn=lambda: input_fn(X_train), steps=TRAIN_EPOCHS)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hy/j_c72d1x72g_rr58tgrlh3b40000gn/T/tmpb6emvn4x', '_tf_random_seed': None, '_save_summary_steps': 1, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c307e1f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 

In [ ]:
predictions_val   = list(estimator_val.predict(input_fn = lambda: input_fn(X_val, pred=True)))
y_preds_val       = [int(x) for x in predictions_val]
mean_absolute_precision_error(y_preds_val, X_val.price)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=make_model, params=hparams, config = rconfig)
estimator.train(input_fn=lambda: input_fn(train_renamed), steps=TRAIN_EPOCHS)

In [ ]:
predictions_val   = list(estimator.predict(input_fn = lambda: input_fn(X_val, pred=True)))
y_preds_val       = [int(x) for x in predictions_val]
mean_absolute_precision_error(y_preds_val, X_val.price)

In [ ]:
predictions   = list(estimator.predict(input_fn = lambda: input_fn(test_renamed, pred=True)))
y_preds_test   = [int(x) for x in predictions]

# Save to File

In [ ]:
filename = "/Users/mohsenkiskani/Downloads/Ubaar/submissions/submission48.csv"
with open(filename,"w+") as outputfile:
    outputfile.write("ID,price\n")
    for i in range(len(y_preds_test)):
        outputfile.write(str(test.ID[i])+","+str(int(np.ceil(y_preds_test[i])))+"\n")

In [ ]:
#hparams = tf.contrib.training.HParams(learning_rate=lr)
#rconfig = tf.estimator.RunConfig(log_step_count_steps = 10)

In [ ]:
#estimator_val = tf.estimator.DNNRegressor(model_fn=make_model, params=hparams, config = rconfig)
#estimator_val.train(input_fn=lambda: input_fn(X_train), steps=TRAIN_EPOCHS)